In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda
import time
import imageio.v2 as imageio
import os

# Параметры модели
Lx, Ly = 1.0, 1.0
alpha = 0.01
Nx, Ny = 500, 500  # Умеренное разрешение
dx, dy = Lx / (Nx - 1), Ly / (Ny - 1)
dx2, dy2 = dx**2, dy**2
dt = 0.5 * (dx2 * dy2) / (2 * alpha * (dx2 + dy2))

T_total = 10.0
Nt = int(T_total / dt)

# Параметры GIF
gif_fps = 40                         # Кадров в секунду
dt_physical = 1 / gif_fps           # Шаг по физическому времени между кадрами
snapshot_interval = int(dt_physical / dt)  # Шаг по итерациям

print(f'dt = {dt:.5e}, Nt = {Nt}, snapshot_interval = {snapshot_interval}')

# Инициализация сетки и начальных условий
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)
u = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly).astype(np.float32)

# CUDA-ядро
@cuda.jit
def solve_kernel(u, u_new, alpha, dt, dx2, dy2):
    i, j = cuda.grid(2)
    if 1 <= i < u.shape[0]-1 and 1 <= j < u.shape[1]-1:
        u_new[i,j] = u[i,j] + alpha * dt * (
            (u[i+1,j] - 2*u[i,j] + u[i-1,j])/dx2 + 
            (u[i,j+1] - 2*u[i,j] + u[i,j-1])/dy2
        )

# Решение и генерация кадров
def solve_and_collect_frames(u, alpha, dt, dx2, dy2, Nt, snapshot_interval):
    d_u = cuda.to_device(u)
    d_u_new = cuda.to_device(u.copy())
    
    threads = (16, 16)
    blocks = (
        (u.shape[0] + threads[0] - 1) // threads[0],
        (u.shape[1] + threads[1] - 1) // threads[1]
    )

    frames = []
    for step in range(Nt):
        solve_kernel[blocks, threads](
            d_u, d_u_new, 
            np.float32(alpha), np.float32(dt),
            np.float32(dx2), np.float32(dy2)
        )
        d_u, d_u_new = d_u_new, d_u

        if step % snapshot_interval == 0:
            u_host = d_u.copy_to_host()
            fig, ax = plt.subplots()
            cf = ax.contourf(X, Y, u_host, levels=100, cmap='viridis')
            plt.colorbar(cf, ax=ax)
            ax.set_title(f"t = {step * dt:.2f} сек")
            ax.set_xlabel("x")
            ax.set_ylabel("y")

            fname = f"frame_{step:05d}.png"
            fig.savefig(fname, dpi=100)
            frames.append(fname)
            plt.close(fig)

    return frames

# Запуск вычислений и создание GIF
start = time.time()
frames = solve_and_collect_frames(u, alpha, dt, dx2, dy2, Nt, snapshot_interval)
end = time.time()
print(f"Генерация {len(frames)} кадров заняла {round(end - start, 2)} сек.")

# Сохранение GIF
images = [imageio.imread(fname) for fname in frames]
imageio.mimsave("heat_equation.gif", images, fps=gif_fps)
print("GIF сохранён как 'heat_equation.gif'.")

# Очистка временных файлов
for fname in frames:
    os.remove(fname)


dt = 5.02006e-05, Nt = 199200, snapshot_interval = 498
Генерация 400 кадров заняла 164.23 сек.
GIF сохранён как 'heat_equation.gif'.
